<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

  Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Implicit
### Example 

In [ ]:
class Rational(val n: Int, val d:Int) {
  require(d != 0)
  override def toString = n + "/" + d
  def +(x: Rational): Rational = new Rational(this.n*x.d + x.n*this.d, this.d * x.d)
  def +(x: Int): Rational = new Rational(this.n + x*this.d, this.d)
}
val a = new Rational(2,3)

In [ ]:
val b = new Rational(1,4)
val r0 = a + b
val r1 = a + 2

In [ ]:
val r2 = 2 + a

This cause an error, you can find I`nt 2` have not method `+(x: Rtional)`

we have 2 way to implement that 

**way1 translate Int to Rational**  
There's nothing to discuss about explicit transformation, This is what we do in general languages
```
val r2 = new Ration(2,1) + a
```
If there is a large number of such operations, it is obviously very cumbersome. so an implicit transform is very elegant

In [ ]:
implicit def transInt2Rational(x: Int) = new Rational(x,1)
val r2 = 2 + a

**way2 add a method of `+(x: Rational)` to Int**  

We can't modify Scala's source code to add methods to Int.  

But Scala's fascinating implicit transformation allows us to add methods to a type without modifying the source code, as if the type itself had its own methods

In [ ]:
implicit class expandInt(t: Int){
    def +(x: Rational): Rational = new Rational(t,1) + x
}
val r2 = 2 + a

## 隐式转换
**当需要的时候就转换，否则啥也不干**


In [ ]:
case class Euro(value: Double)
case class Dollar(value: Double)

object Dollar{
    implicit def dollarToEuro(x: Dollar): Euro = Euro(x.value*0.75)
}
val a = Dollar(12)
val b: Euro = Dollar(10)

In [ ]:
val a = 3.001 * 2

In [ ]:
val b = 2 * 3.0001 

object Int {
 def *(x :Int): Int = this * x
 def *(x :Double): Double = Double(this) * x
}

In [ ]:
val x: Double = 2

In [ ]:
val x:Int = 3.000

In [ ]:
val x: Int = 3.001.toInt

## 隐式扩展
**当需要的时候就转换，否则啥也不干**


## 隐式参数 
**隐式的定义一个参数，并且隐式的传递进去，对用户透明**


In [ ]:
object Test{
    def show(implicit h: Euro) = {
        println(h.value)
    }
}
implicit val h = Euro(12)
Test.show

## 隐式生成（隐式转换和隐式参数的协作）

**作用**：没有的时候生成一个，有了就什么也不干   
**区别**：隐式转换带参数，隐式生成不带参数

*隐式转换并没有输入参数，那么只要需要隐式的对象若没有声明，那么就生成一个，否则什么也不敢*
```scala
def hello(implicit h: ClassName) = {}
...
Test.hello //没有传递函数时会诱导隐式生成生成一个ClassName 对象给它
```


In [ ]:
case class ClassName(name: String)
object ClassName {
  //需要的时候生成一个 
  implicit def genWhenNeed: ClassName = ClassName("Example")
}
object Test{
    def hello(implicit h: ClassName) = {
        println(h.name)
    }
}
Test.hello(ClassName("jack"))
Test.hello

除了上面的那种方法诱导隐式生成发生以外

**还可以直接使用`implicitly[]`关键字**，注意是`[]`， 不是`()`

```scala
val b = implicitly[ClassName]
```

In [ ]:
val b = implicitly[ClassName]

## 隐式转换加宏

隐式转换加持宏可以和编译器打交道
使用中有两种方法：

- 方法一
```scala 
def getLineNumber(implicit line: LineNumber) ={
      println("add at ${line}"
      line
}
val s = getLineNumber
```

- 方法二
```scala
val s = implicitly[LineNumber]
```


(*注： Scala宏在Jupyter执行中会有问题，实例请在sbt环境下编译运行*)


In [ ]:
//获取当前行数
import language.experimental.macros
import reflect.macros.blackbox.Context

case class LineNumber(no: Int)

object LineNumber {
  implicit def genWhenNeed: LineNumber = macro Macros.lineNumberImpl
}

object Macros{
  def lineNumberImpl(c: Context): c.Expr[LineNumber] = {
    import c.universe._
    val lineNumber = c.enclosingPosition.line
    c.Expr[LineNumber](q"""${c.prefix}($lineNumber)""")
  }
}

In [ ]:
//获取变量名
import language.experimental.macros
import reflect.macros.blackbox.Context

case class SymbolName(name: String)
case class ClassName(name: String)

object SymbolName{
  implicit def genWhenNeed: SymbolName = macro Macros.symbolNameImpl
}
 

object Macros{
  def symbolNameImpl(c: Context): c.Expr[SymbolName] = {
    import c.universe._
    val symbolName = c.internal.enclosingOwner.name.decodedName.toString.trim
    c.Expr[SymbolName](q"""${c.prefix}($symbolName)""")
  }
}

## Creat a Complex from Zero
```scala
val a = 2 + 3.2*j 
val b = 3.001 -7*j
val c = a * b
println(c)
28.402 - 4.397j
```

In [ ]:
import scala.math._
case class Complex(re: Double, im: Double) {
    def +(x: Complex): Complex = Complex(re + x.re, im + x.im)
    def -(x: Complex): Complex = Complex(re - x.re, im - x.im)
    def *(x: Double):  Complex = Complex(re * x, im * x)
    def *(x: Complex): Complex = Complex(re * x.re - im * x.im, re * x.im + im * x.re)
    def /(x: Double):  Complex = Complex(re / x, im / x)
 
    override def toString(): String = {
        val a = "%1.3f".format(re)
        val b = "%1.3f".format(abs(im))
        (a,b) match {
            case (_, "0.000") => a
            case ("0.000", _) => b + "j"
            case (_, _) if im > 0 => a + " + " + b + "j"
            case (_, _) => a + " - " + b + "j"
        }
    }
} 

In [ ]:
trait jbase
object j extends jbase
case class image(value: Double)
implicit class compleInt(x: Int) {
    def *(y: jbase) = image(x toDouble)
}
implicit class compleDouble(x: Double) {
    def *(y: jbase) = image(x)
}
implicit class DoubleExpand(x: Double) {
    def +(that: image) = Complex(x,that.value)
    def -(that: image) = Complex(x,-that.value)
}
implicit class intExpand(x: Int) {
    def +(that: image) = Complex(x toDouble,that.value)
    def -(that: image) = Complex(x toDouble,-that.value)
}

In [ ]:
val a = 2 + 3.2*j 
val b = 3.001 -7*j
val c = a * b
println(c)